In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as soup

In [2]:
browser = Browser('chrome')

In [3]:
url = 'https://phish.net/setlists/phish/'
browser.visit(url)
html = browser.html
phish_soup = soup(html, 'html.parser')

In [5]:
dates = phish_soup.find_all('span', class_='setlist-date')

for date in dates:
    print(date)

<span class="setlist-date">
<a href="/setlists/phish">PHISH</a>,
	<a href="https://phish.net/setlists/phish-august-31-2023-dicks-sporting-goods-park-commerce-city-co-usa.html">THURSDAY 08/31/2023</a>
</span>
<span class="setlist-date">
<a href="/setlists/phish">PHISH</a>,
	<a href="https://phish.net/setlists/phish-august-26-2023-broadview-stage-at-spac-saratoga-springs-ny-usa.html">SATURDAY 08/26/2023</a>
</span>
<span class="setlist-date">
<a href="/setlists/phish">PHISH</a>,
	<a href="https://phish.net/setlists/phish-august-25-2023-broadview-stage-at-spac-saratoga-springs-ny-usa.html">FRIDAY 08/25/2023</a>
</span>
<span class="setlist-date">
<a href="/setlists/phish">PHISH</a>,
	<a href="https://phish.net/setlists/phish-august-05-2023-madison-square-garden-new-york-ny-usa.html">SATURDAY 08/05/2023</a>
</span>
<span class="setlist-date">
<a href="/setlists/phish">PHISH</a>,
	<a href="https://phish.net/setlists/phish-august-04-2023-madison-square-garden-new-york-ny-usa.html">FRIDAY 08/

In [ ]:
title_elem = slide_elem.find('href', class_='content_title')
print(title_elem)